Install prequisite packages

In [ ]:
!pip install --upgrade azureml-sdk opencv-contrib-python

In [ ]:
import azureml.core

print("This notebook was created using version 1.35.0 of the Azure ML SDK.")
print("You are currently using version", azureml.core.VERSION, "of the Azure ML SDK.")
assert (
    azureml.core.VERSION >= "1.35"
), "Please upgrade the Azure ML SDK by running '!pip install --upgrade azureml-sdk' then restart the kernel."

In [ ]:
!pip install azure-core azure-storage-blob azure-storage-file-datalake==12.5.0 

In [ ]:
import os
import time
import numpy as np
import cv2
from PIL import Image
from datetime import datetime
# from azure.storage.filedatalake._models import ContentSettings
# from azure.storage.filedatalake import DataLakeServiceClient
from azure.storage.blob import BlobServiceClient, ContainerClient, BlobClient 
from azure.core.exceptions import ResourceExistsError

In [ ]:
# Variable inputs
file_name = "your file name"
file_path = "your_path_to_file"
full_file_path = os.path.join(file_path, file_name)
frame_count = 0

In [ ]:
# Create ADL G2 or Blob Storage Client
fs_name = "your_file_system_or_blob_name"
dir_name = os.path.basename(full_file_path).split('.')[0]
dir_name = str.lower(dir_name)
# adl_g2_conn_str = "your_ADL_G2_connection_string"
blob_conn_str = "your_blob_connection_string"
# storage_client = DataLakeServiceClient.from_connection_string(adl_g2_conn_str)
storage_client = BlobServiceClient.from_connection_string(blob_conn_str)

In [ ]:
#  Create Blob Container
def create_container(base_dir):
    global container_client
    try:
        container_client = storage_client.create_container(base_dir)
    
    except ResourceExistsError as e:
        # print(e)
        container_client = storage_client.get_container_client(base_dir)

create_container(fs_name)

In [ ]:
# Create ADL G2 File System (base container)
# def create_file_system(base_dir):
#     global file_system_client
#     try:

#         file_system_client = storage_client.create_file_system(file_system=base_dir)
    
#     except ResourceExistsError as e:

#         file_system_client = storage_client.get_file_system_client(base_dir)

# # Create directory in ADL G2
# def create_directory(target_dir):
#     global directory_client
#     try:
#         directory_client = file_system_client.create_directory(target_dir)
    
#     except ResourceExistsError as e:

#         directory_client = file_system_client.get_directory_client(target_dir)

# create_file_system(fs_name)
# create_directory(dir_name)

In [ ]:
# def cap_video_file():
vid_cap = cv2.VideoCapture(full_file_path)
if not vid_cap.isOpened():
    print('Cannot open file')

# Capture individual frames
while(vid_cap.isOpened()):
    _, frame = vid_cap.read()
    if _ == True:
        frame_count += 1
        # Show frame size
        h, w = frame.shape[:2]
        print("\n\n Frame height = {} \n Frame width = {} \n\n ".format(h,w))
        
        # create naming structure
        now = datetime.now()  
        filetime = now.strftime("%Y%d%m%H%M%S%f")
        frame_num = str(frame_count).zfill(6)
        frame_file_name = f"Frame{frame_num}-{filetime}-raw.jpg"
        frame_encoded = cv2.imencode(".jpg", frame)[1].tobytes()
        
        # ADL G2 code
        # try:
        #     file_client = directory_client.create_file(frame_file_name)
        #     file_client.append_data(data=frame_encoded, offset=0, length=len(frame_encoded))
        #     file_client.flush_data(len(frame_encoded))
        #     print(f"Uploaded image: {frame_file_name}")

        # except Exception as e:
        #     print(e)

        # Blob code 
        try:
            blob_client = container_client.get_blob_client(f"{dir_name}/{frame_file_name}")       
            blob_client.upload_blob(frame_encoded, blob_type="BlockBlob")
            print(f"Successfully uploaded {frame_file_name}")
        except Exception as e:
            print(e)

